#### [1] 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/data.csv')

In [3]:
# 데이터 크기 확인
df.shape

(5842, 2)

In [4]:
# 데이터 살펴보기
df.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral


#### [2] 데이터 전처리

[2 - 0] 라벨 인코딩 

In [5]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df['Sentiment'])
label_encoded = le.transform(df['Sentiment'])

In [6]:
# 인코딩된 열 생성
df['label'] = label_encoded

In [7]:
df['label'].value_counts()

1    3130
2    1852
0     860
Name: label, dtype: int64

[2 - 1] 데이터 전처리

사전 만들기

In [8]:
import re
# 표제어 추출은 일단 하지 말자
# from nltk.stem import LancasterStemmer
# stemmer = LancasterStemmer()

In [9]:
def clean_text(df,field):
    df[field] = df[field].str.replace(r"http\S+","")
    df[field] = df[field].str.replace(r"http","")
    df[field] = df[field].str.replace(r"@","at")
    df[field] = df[field].str.replace("#[A-Za-z0-9_]+", '')
    df[field] = df[field].str.replace(r"[^A-Za-z(),!?@\'\"_\n]"," ")
    df[field] = df[field].str.lower()
    return df 

In [10]:
def preprocess_text(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", "not", text)
    text = re.sub(r"\'re", "are", text)
    text = re.sub(r"\'s", "is", text)
    text = re.sub(r"\'d", "would",text)
    text = re.sub(r"\'ll", "will", text)
    text = re.sub(r"\'t", "not", text)
    text = re.sub(r"\'ve", "have", text)
    text = re.sub(r"\'m", "am", text)
    text = re.sub('[^a-zA-Z]',' ',text)
    text = re.sub(r'[^\x00-\x7f]','',text)
    return text

In [11]:
cleaned_df = clean_text(df, "Sentence")

C:\Users\LJH\AppData\Local\Temp\ipykernel_11300\267789293.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df[field] = df[field].str.replace(r"http\S+","")
C:\Users\LJH\AppData\Local\Temp\ipykernel_11300\267789293.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df[field] = df[field].str.replace("#[A-Za-z0-9_]+", '')
C:\Users\LJH\AppData\Local\Temp\ipykernel_11300\267789293.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df[field] = df[field].str.replace(r"[^A-Za-z(),!?@\'\"_\n]"," ")


In [12]:
cleaned_df["Sentence"] = cleaned_df["Sentence"].apply(preprocess_text)

In [13]:
df['Sentence'][0]

'the geosolutions technology will leverage benefon is gps solutions by providing location based search technology   a communities platform   location relevant multimedia content and a new and powerful commercial model  '

In [14]:
df.head()

,Sentence,Sentiment,label
0,the geosolutions technology will leverage bene...,positive,2
1,esi on lows down to bk a real po...,negative,0
2,for the last quarter of componenta is n...,positive,2
3,according to the finnish russian chamber of co...,neutral,1
4,the swedish buyout firm has sold its remaining...,neutral,1


In [35]:
# 토큰화
from nltk.tokenize import word_tokenize

# 토큰들을 담을 리스트 생성
list_tokens_00 = []

# 전체 단어를 담을 리스트 생성
for i in range(0, len(df['Sentence'])) :
    result = word_tokenize(df['Sentence'][i])
    for j in range(0,len(result)) :
        list_tokens_00.append(result[j])

In [36]:
list_word_bag = []
# 전체 토큰화 갯수
list_word_bag.append(len(list_tokens_00))
print(len(list_tokens_00))

104860


In [37]:
# 중복된 토큰 제거
set_tokens = set(list_tokens_00)
list_tokens_01 = list(set_tokens)
list_word_bag.append(len(list_tokens_01))
print(len(list_tokens_01))

10123


In [39]:
# 불용어 제거
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english')) 

list_tokens_02 = []

for word in list_tokens_01 :
    if word not in stop_words :
        list_tokens_02.append(word)

In [40]:
list_word_bag.append(len(list_tokens_02))
print(len(list_tokens_02))

9998


문장 전처리

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

In [43]:
ctv = CountVectorizer(tokenizer= word_tokenize, token_pattern=None)

ctv.fit_transform(df['Sentence'])

<5842x10123 sparse matrix of type '<class 'numpy.int64'>'
	with 93419 stored elements in Compressed Sparse Row format>

In [44]:
ctv

CountVectorizer(token_pattern=None,
                tokenizer=<function word_tokenize at 0x000001E2AC45D790>)